In [1]:
import os
from langchain.docstore.document import Document

os.environ['OPENAI_API_KEY'] = "API KEY HERE"

class MarkdownLoader:
    def __init__(self, directory):
        self.directory = directory

    def load(self):
        documents = []
        for root, _, files in os.walk(self.directory):
            for file in files:
                if file.endswith('.md'):
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r') as f:
                        content = f.read()
                        metadata = {'source': file_path}
                        documents.append(Document(page_content=content, metadata=metadata))
        return documents

In [2]:
# Load your documents
# !git clone https://github.com/polywrap/documentation.git #uncomment this line the first time code is run
loader = MarkdownLoader("documentation")
docs = loader.load()

Cloning into 'documentation'...
remote: Enumerating objects: 16310, done.
remote: Counting objects: 100% (1371/1371), done.
remote: Compressing objects: 100% (625/625), done.
remote: Total 16310 (delta 563), reused 1007 (delta 393), pack-reused 14939
Receiving objects: 100% (16310/16310), 22.86 MiB | 14.06 MiB/s, done.
Resolving deltas: 100% (8861/8861), done.


In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain


text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

Created a chunk of size 2012, which is longer than the specified 1000
Created a chunk of size 1583, which is longer than the specified 1000
Using embedded DuckDB without persistence: data will be transient


In [4]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [5]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

In [6]:
query = "How can I make a polywrap call to a hello world wrapper?"
result = qa({"question": query})

In [7]:
result

{'question': 'How can I make a polywrap call to a hello world wrapper?',
 'chat_history': [HumanMessage(content='How can I make a polywrap call to a hello world wrapper?', additional_kwargs={}, example=False),
  AIMessage(content=' You can make a polywrap call to a hello world wrapper by using the Polywrap JavaScript Client. You can install the client with the command `npm install --save @polywrap/client-js`. Then, you can invoke the wrapper with the following code: \n\n```javascript\nconst result = await client.invoke({\n  uri: "ens/wraps.eth:logger@1.0.0",\n  method: "log",\n  args: {\n    message: "Hello Polywrap!",\n  },\n});\n\nconsole.log(result);\n```\n\nRunning the application using `node index.js`, you should now see two lines appear in your console:\n\n```\nHello Polywrap!\n{ ok: true, value: true }\n```\n\nThe first line is printed by the Logger Wrap, while the second line shows the structure of the `InvokeResult` object.', additional_kwargs={}, example=False)],
 'answer': '